# Analyzing Thankgsgiving data of US-households

This notebook was built during the e-learning course of dataquest.io. The notebook analyzes 1058 responses on an online survey which deals with the Thanksgiving dinner of Americans. The dataset contains 65 columns which firstly save information about the dinner habits of the people and secondly save informationen about democgraphical data.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("thanksgiving.csv", encoding="Latin-1")
data.head(1)

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic


In [4]:
data.columns

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

## Remove Answers of people who dont celebrate Thanksgiving

In [6]:
data["Do you celebrate Thanksgiving?"].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [7]:
data = data[data["Do you celebrate Thanksgiving?"] == "Yes"]
data["Do you celebrate Thanksgiving?"].value_counts()

Yes    980
Name: Do you celebrate Thanksgiving?, dtype: int64

## Analyzing what people prefer to eat at Thanksgiving

In [9]:
data["What is typically the main dish at your Thanksgiving dinner?"].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [10]:
column = "What is typically the main dish at your Thanksgiving dinner?"
toForkeyDinners = data[data[column] == "Tofurkey"]
toForkeyDinners["Do you typically have gravy?"]

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object

## Analyzing what people prefer for dessert

In [6]:
apple_column = "Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"
apple_isnull = pd.isnull(data[apple_column])
pumpkin_column = "Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"
pumpkin_isnull = pd.isnull(data[pumpkin_column])
pecan_column = "Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"
pecan_isnull = pd.isnull(data[pecan_column])

In [13]:
ate_pies = (apple_isnull & pumpkin_isnull & pecan_isnull)
ate_pies.value_counts()

False    876
True     182
dtype: int64

## Analyzing age of people who participated in the thankgsgiving survey

In [8]:

def getAge(string):
    if pd.isnull(string):
        return None 
    else:
        newString = string.split(" ")[0]
        newString = newString.replace("+", "")
        return int(newString)

data["int_age"] = data["Age"].apply(getAge)
data["int_age"].describe()


count    1025.000000
mean       39.383415
std        15.398493
min        18.000000
25%        30.000000
50%        45.000000
75%        60.000000
max        60.000000
Name: int_age, dtype: float64

## Findings: 
As we are working with a range of ages not an exact age the problem with the implemented solution is that we are only looking at the lower bound.

## Analyzing income of survey participants

In [4]:
def getIncome(string):
    if pd.isnull(string):
        return None
    else:
        newString = string.split(" ") [0]
        if newString == "Prefer":
            return None
        else:
            newString = newString.replace("$","")
            newString = newString.replace(",","")
            return int(newString)

data["int_income"] = data["How much total combined money did all members of your HOUSEHOLD earn last year?"].apply(getIncome)
data["int_income"].describe()

count       889.000000
mean      74077.615298
std       59360.742902
min           0.000000
25%       25000.000000
50%       50000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

As we are working with a range again and only looking at the lower bound the results need to be critically questioned. Never the less the income of the participants looks quite high but the STD is also kind of high

## Analyzing travel distance of people connected to income
Assumption: People with less money are younger and travel to their parents house for thanksgivinv

In [6]:
incomeUnder150k = data[data["int_income"] < 150000]["How far will you travel for Thanksgiving?"]
incomeUnder150k.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [7]:
incomeUnder150k = data[data["int_income"] > 150000]["How far will you travel for Thanksgiving?"]
incomeUnder150k.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64

In [11]:
ageUnder45 = data[data["int_age"] < 45]["How far will you travel for Thanksgiving?"]
ageUnder45.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         149
Thanksgiving is local--it will take place in the town I live in                     132
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    104
Thanksgiving is out of town and far away--I have to drive several hours or fly       35
Name: How far will you travel for Thanksgiving?, dtype: int64

In [12]:
ageOver45 = data[data["int_age"] > 45]["How far will you travel for Thanksgiving?"]
ageOver45.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         123
Thanksgiving is local--it will take place in the town I live in                      72
Thanksgiving is out of town but not too far--it's a drive of a few hours or less     47
Thanksgiving is out of town and far away--I have to drive several hours or fly       16
Name: How far will you travel for Thanksgiving?, dtype: int64

## Findings
First of all and not surprisingly there are much more people with income under 150k than over. And most of the people dont travel for thankgsgiving. The amount of people traveling a long distance is very small.

## Analyzing Friendship Thanksgiving and Age

In [16]:
data.pivot_table(index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",
                 columns='Have you ever attended a "Friendsgiving?"', 
                 values="int_age")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [20]:
data.pivot_table(index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",
                 columns='Have you ever attended a "Friendsgiving?"', 
                 values="int_income")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,78914.549654,72894.736842
Yes,78750.000000,66019.736842
